In [26]:
import json
import os
from omero.rtypes import rstring
from omero.gateway import BlitzGateway
from biohack_utils.config_utils import flatten, unflatten, download, upload
from biohack_utils.ConfigSchema import OMEWrapper
from biohack_utils.ConfigSchema import CollectionNode
from biohack_utils.util import connect_to_omero
import dotenv
import pprint


In [29]:
def connect_to_omero():
    dotenv.load_dotenv()
    USERNAME = os.getenv("OMERO_USERNAME")
    PASSWORD = os.getenv("OMERO_PASSWORD")
    HOST = "omero-training.gerbi-gmb.de"
    PORT = 4064  # Default OMERO port

    conn = BlitzGateway(USERNAME, PASSWORD, host=HOST, port=PORT)
    conn.connect()

    if conn.isConnected():
        print("Connected to OMERO")
    else:
        print("Failed to connect")
        exit(1)

    return conn


In [8]:
dotenv.load_dotenv()
    # Read the json file
json_file_path = "example.json"
    
with open(json_file_path) as f:
    d = json.load(f)

In [9]:
wrapper = OMEWrapper.model_validate(d)

In [10]:
data_dict = wrapper.model_dump()

# 3. Export with Indentation 💾
file_path = "output_from_wrapper.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(data_dict, f, indent=2)

In [17]:
flat = flatten(wrapper)
pprint.pprint(flat)

[{'category': 'intensities',
  'description': 'Raw cell image from CovidIF dataset',
  'omero:image_id': 35501,
  'origin': 'raw',
  'path': 'CovidIF_Cell_Raw'},
 {'category': 'intensities',
  'description': 'Raw nucleus image from CovidIF dataset',
  'omero:image_id': 35502,
  'origin': 'raw',
  'path': 'nucleus_analysis/CovidIF_Nucleus_Raw'},
 {'category': 'annotations',
  'description': 'Nucleus segmentation from CovidIF dataset',
  'omero:image_id': 35503,
  'origin': 'masks',
  'path': 'nucleus_analysis/nucleus',
  'source': 'CovidIF_Nucleus_Raw'}]


In [18]:
# Unflatten example
unflattened = unflatten(flat, "cell_analysis").model_dump()
# 3. Export with Indentation 💾
file_path = "output_from_after_unwrapping.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(unflattened, f, indent=2)

In [19]:
conn = connect_to_omero()

Connected to OMERO


In [20]:
coll_id = upload(conn,wrapper)

In [22]:
schema = download(conn, 11104)


Processing image 35501
  Found 2 node annotations via query
  ✓ Match found!

Processing image 35502
  Found 2 node annotations via query
  ✓ Match found!

Processing image 35503
  Found 2 node annotations via query
  ✓ Match found!

Total flat records collected: 3


In [16]:
schema

# 3. Export with Indentation 💾
file_path = "output_from_omero.json"

with open(file_path, 'w', encoding='utf-8') as f:
    # Key Step: Use json.dump() and set indent=4
    # The 'indent=4' parameter tells Python to add line breaks and 4 spaces
    # of indentation for every nested level.
    json.dump(data_dict, f, indent=2)

In [ ]:
_get_node_info(conn, image_id, collection_id=None)


AttributeError: 'NoneType' object has no attribute 'listAnnotations'